### 热身

In [1]:
data = [1, 2, 3, 4, 5, 6]

In [2]:
res1 = []
for n in data:
    res1.append(n * 2)
print(res1)


[2, 4, 6, 8, 10, 12]


In [3]:
res2 = [n*2 for n in data]
print(res2)

[2, 4, 6, 8, 10, 12]


In [4]:
def doubled(n):
    return n * 2

res3 = map(doubled, data)
res3 = list(res3)
print(res3)

[2, 4, 6, 8, 10, 12]


### 分式化简

In [5]:
import random

In [6]:
data = [(random.randint(1,1000), random.randint(1,1000)) for _ in range(100000)]

In [7]:
def gcd(a, b):
    r = a % b
    while r != 0:
        a = b
        b = r
        r = a % b
    return b

def simplify(pair):
    n1, n2 = pair[0], pair[1]
    m = gcd(n1, n2)
    return (int(n1/m), int(n2/m))

In [8]:
%%time

res1 = []
for pair in data:
    n1, n2 = pair[0], pair[1]
    m = gcd(n1, n2)
    res1.append((int(n1/m), int(n2/m)))
    
#print(res1)

CPU times: user 58.5 ms, sys: 24.5 ms, total: 83 ms
Wall time: 83.8 ms


In [9]:
%%time
res2 = [simplify(pair) for pair in data]
#print(res2)

CPU times: user 76.4 ms, sys: 0 ns, total: 76.4 ms
Wall time: 78.4 ms


In [10]:
%%time
res3 = map(simplify, data)
res3 = list(res3)

CPU times: user 78 ms, sys: 0 ns, total: 78 ms
Wall time: 80 ms


### 多进程

In [11]:
from multiprocessing import Pool  # 理论上这个就可以用，但是在Windows上会卡住，细节不在这里研究
#from multiprocessing.pool import ThreadPool as Pool  # 使用线程
#from multiprocessing.dummy import Pool  # 用线程模拟进程

In [12]:
p = Pool(4)  # 创建一个进程池，开4个线程

In [13]:
%%time
res4 = p.map(simplify, data)

CPU times: user 1.22 ms, sys: 51.7 ms, total: 52.9 ms
Wall time: 76.4 ms


In [14]:
# 这个时间测试不太准，但理论是这样的

In [15]:
print(res3[:5], res4[:5])

[(153, 26), (256, 367), (52, 95), (977, 268), (155, 63)] [(153, 26), (256, 367), (52, 95), (977, 268), (155, 63)]
